In [1]:
import numpy as np,os,sys,inspect
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (cross_val_score, RepeatedStratifiedKFold, GridSearchCV, 
                                     StratifiedShuffleSplit, RandomizedSearchCV, ShuffleSplit)
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, matthews_corrcoef
import matplotlib.pyplot as plt
from utils import corrupt_label, load_mat, noisy_evaluate, Logit

This implementation performs better compared to loss_log sometimes, and sometimes worse. Problem is the margin tends to be pretty wide in both cases. I think, but not absolutely certain, that this is correct, complete implementation.

In [ ]:
def compute(clf,X,y,PN,NP):
    scores = []
    for _ in range(5):
        yn = create_noise(y,PN=PN,NP=NP)
        CV = ShuffleSplit(n_splits=3,test_size=.33)
        r = noisy_evaluate(clf,X,yn,y,CV,[accuracy_score])
        scores.append(list(r.values())[0]*100)
        print(list(r.values())[0]*100)
    return round(sum(scores)/len(scores),3)

In [ ]:
params = {
    'PN':np.linspace(.1,.40,7),
    'NP':np.linspace(.1,.40,7),
    'C':np.logspace(-4,4,10)
}
lr = Logit(max_iter=400)
grid_cv = StratifiedShuffleSplit(n_splits=2,test_size=.25)
robust_lr = RandomizedSearchCV(lr,params,n_iter=20,cv=grid_cv,n_jobs=3)
#clf = GridSearchCV(lr,params,cv=StratifiedShuffleSplit(n_splits=3))

In [2]:
datasets = ['heart','german','breast_cancer','diabetis','thyroid','image','banana']
#datasets = ['heart','image']
PN = .4
NP = .4
for d in datasets:
    X,y = load_mat(d) 
    print(d,X.shape,np.unique(y,return_counts=True))
    res = compute(clf,X,y,PN,NP)
    print(res)
    print()
    

heart (270, 13) (array([0, 1], dtype=int32), array([150, 120]))


## Imbalanced - JIRA Datasets

In [ ]:
from libs import read_jira
datasets = [f for f in os.listdir("JIRA/") if 'csv' in f]
datasets = np.random.choice(datasets,size=5)

In [ ]:
grid_cv = StratifiedShuffleSplit(n_splits=2,test_size=.25)
lr = GridSearchCV(LogisticRegression(max_iter=800),{'C':np.logspace(-4,4,10)},cv=grid_cv,n_jobs=3)
SCORERS = [accuracy_score,f1_score,matthews_corrcoef]
CV = StratifiedShuffleSplit(n_splits=10,test_size=.20)

In [ ]:
for d in datasets:
    X,yn,y = read_jira(d)
    cm = confusion_matrix(y,yn)
    rlr = noisy_evaluate(lr,X,yn,y,CV,SCORERS)
    rrobust = noisy_evaluate(robust_lr,X,yn,y,CV,SCORERS)
    for m in SCORERS:
        print(m.__name__)
        print("LR:",rlr[m])
        print("Robust:",rrobust[m])
    print()

In [ ]:
%%time
robust_lr.fit(X,yn)